## Script até aqui

In [ ]:
caminho_dados = '/content/drive/MyDrive/Curso AD/dados_viagens/2023_Viagem.csv'

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Lendo os dados
df_viagens = pd.read_csv(caminho_dados, encoding='Windows-1252', sep=';', decimal=',')

# Criando nova coluna de despesas
df_viagens['Despesas'] = df_viagens['Valor diárias'] + df_viagens['Valor passagens'] + df_viagens['Valor outros gastos']

# Ajustando valores nulos na coluna de cargo
df_viagens['Cargo'] = df_viagens['Cargo'].fillna('NÃO IDENTIFICADO')

# Convertendo colunas de datas
df_viagens['Período - Data de início'] = pd.to_datetime(df_viagens['Período - Data de início'], format='%d/%m/%Y')
df_viagens['Período - Data de fim'] = pd.to_datetime(df_viagens['Período - Data de fim'], format='%d/%m/%Y')

# Criando novas colunas de datas
df_viagens['Mês da viagem'] = df_viagens['Período - Data de início'].dt.month_name()
df_viagens['Dias de viagem'] = (df_viagens['Período - Data de fim'] - df_viagens['Período - Data de início']).dt.days

In [ ]:
df_viagens

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor devolução,Valor outros gastos,Despesas,Mês da viagem,Dias de viagem
0,17821923,000001/23-1C,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26352,Fundação Universidade Federal do ABC,***.875.238-**,PRISCILA LEAL DA SILVA,PROFESSOR DO MAGISTERIO SUPERIOR,-1,Sem informação,2023-01-01,2023-12-02,Loughborough/Reino Unido,Intercâmbio acadêmico PCDP anterior 07/21-3C,0.00,0.00,0.00,0.00,0.00,January,335
1,18236583,000018/23,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26236,Universidade Federal Fluminense,NaN,CAMILLA DUARTE DA SILVA,BIBLIOTECARIO-DOCUMENTALISTA,-1,Sem informação,2023-01-16,2024-02-28,Rio de Janeiro/RJ,Afastamento para pós-graduação stricto sensu n...,0.00,0.00,0.00,0.00,0.00,January,408
2,18288418,000007/23-1C,Realizada,SIM,Por necessidade do serviço.,52000,Ministério da Defesa,52121,Comando do Exército,***.621.358-**,ANDRE LUIS COSTA PITANGUEIRA,NÃO IDENTIFICADO,OfSuperior,OfSuperior,2023-01-10,2023-01-29,Nova York/Estados Unidos da América,Participar da revisão do COE Manual (Contingen...,39565.70,7434.13,0.00,1095.35,48095.18,January,19
3,18296348,000070/23,Realizada,SIM,A efetivação do empenho no Siafi e a respectiv...,26000,Ministério da Educação,26406,Instituto Federal do Espírito Santo,***.407.547-**,LIDIANY MIRANDA FERRAZ NUNES,AUXILIAR DE BIBLIIOTECA,-1,Sem informação,2023-02-05,2023-02-10,Vitória/ES,Participação das aulas de Mestrado Interinstit...,1013.96,0.00,0.00,0.00,1013.96,February,5
4,18302983,000001/23,Realizada,NÃO,Sem informação,52000,Ministério da Defesa,52111,Comando da Aeronáutica,***.855.388-**,GREGOR GASPAR,NÃO IDENTIFICADO,OfIntermed,OfIntermed,2023-01-01,2023-07-07,Gotemburgo/Suécia,PARTICIPAR DO CURSO DE PILOTO DE GRIPEN C/D.,0.00,14903.10,0.00,0.00,14903.10,January,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825756,2023001618,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.748.269-**,BIANCA VIANA KIVEL,ANALISTA DO BANCO CENTRAL,-1,Sem informação,2023-12-05,2023-12-06,Sem informação,Participar de tratativas nos locais das reuniõ...,606.89,3331.29,0.00,0.00,3938.18,December,1
825757,2023001619,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.231.806-**,LUCAS ALVES FREIRE,PROCURADOR DO BANCO CENTRAL,-1,Sem informação,2023-12-11,2023-12-13,Sem informação,Participar da reunião do COPOM .,1089.00,1837.18,0.00,0.00,2926.18,December,2
825758,2023001628,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.632.011-**,SERGIO BOTELHO,TECNICO DO BANCO CENTRAL,-1,Sem informação,2023-12-07,2023-12-08,Sem informação,Acompanhamento do Presidente do BC - São Paulo,734.13,4555.95,0.00,0.00,5290.08,December,1
825759,2023001631,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.407.445-**,NELSON CAMPOS REIS JUNIOR,TECNICO DO BANCO CENTRAL,-1,Sem informação,2023-12-11,2023-12-11,Sem informação,Acompanhamento do Presidente - São Paulo.,319.28,3745.53,0.00,400.00,4464.81,December,0


## Combinando agregações
No caso em que usamos o `groupby`, atuamos apenas em uma única coluna, `"Despesas"`.

Contudo, frequentemente vamos querer dados agrupados de **diferentes colunas**.

Nestes casos, podemos usar o método `.agg` para realizar operações agregadas em colunas específicas:
- `despesa_media`: aplicar método "mean" na coluna "Despesas"
- `duracao_media`: aplicar método "mean" na coluna "Dias de viagem"


In [ ]:
df_viagens.groupby('Cargo').agg(despesa_media=('Despesas', 'mean'), duracao_media=('Dias de viagem', 'mean'))

,despesa_media,duracao_media
Cargo,,
AAD-AUX DE RECURSOS MATERIAIS,3592.10,9.40
AAD-AUXILIAR ADMINISTRATIVO,115.76,0.00
AAD-AUXILIAR DE OPERACOES,7973.39,16.71
AAD-AUXILIAR DE RECURSOS FINANCEIROS,15545.94,5.50
AAD-OPERADOR DE MICROCOMPUTADOR,1186.47,4.00
...,...,...
VICE-PRESIDENTE DA REPUBLICA,7068.20,1.50
VIDREIRO,0.00,6.00
VIGILANTE,693.56,1.78


Vamos adicionar mais colunas de agregação:
- `despesas_totais`: aplicar método "sum" na coluna "Despesas"
- `destino_mais_frequente`: aplicar método "pd.Series.mode" na coluna "Destinos"
- `n_viagens`: aplicar método "count" em qualquer coluna (contagem simples)

**Dica**: use parênteses para quebrar a operação em múltiplas linhas, facilita a leitura!

In [ ]:
(df_viagens
 .groupby('Cargo')
 .agg(
    despesa_media=('Despesas', 'mean'),
    duracao_media=('Dias de viagem', 'mean'),
    despesas_totais=('Despesas', 'sum'),
    destino_mais_frequente=('Destinos', pd.Series.mode),
    n_viagens=('Nome', 'count')
    )
 .reset_index()
 .sort_values(by='despesas_totais', ascending=False))

,Cargo,despesa_media,duracao_media,despesas_totais,destino_mais_frequente,n_viagens
494,NÃO IDENTIFICADO,3261.98,8.02,1047014709.49,Brasília/DF,320975
443,Informações protegidas por sigilo,3125.72,7.37,369297630.96,Informações protegidas por sigilo,118148
569,PROFESSOR DO MAGISTERIO SUPERIOR,2031.42,3.83,110699956.30,Brasília/DF,54494
570,PROFESSOR ENS BASICO TECN TECNOLOGICO,1401.33,2.34,52454765.16,Brasília/DF,37432
686,TECNICO DO SEGURO SOCIAL,4299.22,11.36,45756570.47,Brasília/DF,10643
...,...,...,...,...,...,...
734,TECNICO EM METALURGIA,0.00,3.00,0.00,"[Alumínio/SP, Maceió/AL]",2
454,LOCUTOR,0.00,7.00,0.00,"[Catuti/MG, Catuti/MG, Catuti/MG, Catuti/MG, C...",2
566,PROFESSOR ADJUNTO,0.00,5.00,0.00,Buenos Aires/Argentina,1
818,VIDREIRO,0.00,6.00,0.00,Brasília/DF,1


Vamos passar essa tabela consolidada para uma variável:

In [ ]:
df_viagens_consolidado = (df_viagens
 .groupby('Cargo')
 .agg(
    despesa_media=('Despesas', 'mean'),
    duracao_media=('Dias de viagem', 'mean'),
    despesas_totais=('Despesas', 'sum'),
    destino_mais_frequente=('Destinos', pd.Series.mode),
    n_viagens=('Nome', 'count')
    )
 .reset_index()
 .sort_values(by='despesas_totais', ascending=False))

## Aplicando um filtro na agregação

Os dados consolidados estão ok, mas incluem diversos cargos que tem pouca representação no conjunto de dados geral.

Vamos filtrar as linhas cujos cargos correspondem a menos de 1% das viagens dessa tabela agregada:

In [ ]:
df_viagens['Cargo'].value_counts(normalize=True).reset_index()

,Cargo,proportion
0,NÃO IDENTIFICADO,0.39
1,Informações protegidas por sigilo,0.14
2,PROFESSOR DO MAGISTERIO SUPERIOR,0.07
3,PROFESSOR ENS BASICO TECN TECNOLOGICO,0.05
4,CONTRATADO LEI 8745/93 - NI,0.02
...,...,...
817,TECNICO EM HIGIENE DENTAL,0.00
818,OPERADOR DE TELE IMPRESSORA,0.00
819,ASSISTENTE TECNICO EM DESENV REGIONAL,0.00
820,TECNICO DE MANUTENCAO,0.00


Temos que pegar a coluna `Cargo` nas linhas onde a coluna `proportion` é maior que 0.01.

Isso é essencialmente um PROCV no Excel, que vamos fazer em Python de forma muito mais simples com `df.loc`!

Com esse método, passamos um seletor de linha(s) e um seletor de coluna(s) separados por vírgula:

`df.loc[ LINHAS_PARA_SELECIONAR, COLUNAS_PARA_SELECIONAR ]`

In [ ]:
df_cargos = df_viagens['Cargo'].value_counts(normalize=True).reset_index()
df_cargos.loc[df_cargos['proportion'] > 0.01, 'Cargo']

,Cargo
0,NÃO IDENTIFICADO
1,Informações protegidas por sigilo
2,PROFESSOR DO MAGISTERIO SUPERIOR
3,PROFESSOR ENS BASICO TECN TECNOLOGICO
4,CONTRATADO LEI 8745/93 - NI
5,AUDITOR FISCAL FEDERAL AGROPECUARIO
6,ANALISTA AMBIENTAL
7,ASSISTENTE EM ADMINISTRAçãO
8,TECNICO INFOR GEOGRAFICAS E ESTATISTICA
9,TECNICO DO SEGURO SOCIAL


Agora, podemos filtrar o df_viagens_consolidado (tabela anterior) pelos cargos que aparecem no resultado acima.

Para fazer isso, vamos usar o método `.isin()`:

In [ ]:
cargos_relevantes = df_cargos.loc[df_cargos['proportion'] > 0.01, 'Cargo']

filtro = df_viagens_consolidado['Cargo'].isin(cargos_relevantes)
filtro

,Cargo
494,True
443,True
569,True
570,True
686,True
...,...
734,False
454,False
566,False
818,False


O resultado é uma série booleana, que já sabemos que dá para usar como filtro!

In [ ]:
df_viagens_consolidado_filtrado = df_viagens_consolidado[filtro_cargos_relevantes]
df_viagens_consolidado_filtrado

,Cargo,despesa_media,duracao_media,despesas_totais,destino_mais_frequente,n_viagens
494,NÃO IDENTIFICADO,3261.98,8.02,1047014709.49,Brasília/DF,320975
443,Informações protegidas por sigilo,3125.72,7.37,369297630.96,Informações protegidas por sigilo,118148
569,PROFESSOR DO MAGISTERIO SUPERIOR,2031.42,3.83,110699956.30,Brasília/DF,54494
570,PROFESSOR ENS BASICO TECN TECNOLOGICO,1401.33,2.34,52454765.16,Brasília/DF,37432
686,TECNICO DO SEGURO SOCIAL,4299.22,11.36,45756570.47,Brasília/DF,10643
72,ANALISTA AMBIENTAL,2595.93,5.14,35302045.45,Brasília/DF,13599
209,AUDITOR-FISCAL DA RECEITA FEDERAL BRASIL,2418.70,2.84,24399834.64,Brasília/DF,10088
208,AUDITOR FISCAL FEDERAL AGROPECUARIO,1465.85,2.06,24306782.30,Brasília/DF,16582
315,CONTRATADO LEI 8745/93 - NI,984.45,2.84,17510463.07,Bauru/SP,17787
776,TECNICO INFOR GEOGRAFICAS E ESTATISTICA,1457.08,3.19,15816641.46,Brasília/DF,10855


Esse tipo de lógica complexa é o que Python te permite fazer com poucas linhas, e de forma automática para qualquer outra tabela no mesmo formato.